In [1]:
import pandas as pd
import datadotworld as ddw
import re

In [2]:
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 70)

The code in this notebook will parse the `address` column in the `accounts_address` file and create two new columns: `zip_code` and `state_abr`. It uses the python SDK to download the data and to upload the derived data file.

### Load the customer info data file from data.world for data cleaning
Enter the dataset to download the data from     
eg, `dataset = 'ddw-biz-team/guided-eval'`

In [ ]:
# specify location of dataset on data.world
dataset = ''

In [3]:
# download data
customer_data = ddw.load_dataset(dataset, force_update=True)
# load data into Python as Pandas dataframe
customer_df = customer_data.dataframes['accounts_address']

In [4]:
print("dataframe shape: ", customer_df.shape)
# view first 3 rows
customer_df.head(3)

dataframe shape:  (97, 4)


,account,revenue,employees,address
0,Sunnamplex,4592.96,13938.0,"Apt. 209 94528 Carleton Route, Jaidenchester, IL 74994-0588"
1,Silis,5339.57,18053.0,"Suite 103 106 Franecki Pines, Port Genesis, IL 28566"
2,Groovestreet,2728.86,6486.0,"1647 Auer Views, Dinabury, IL 44654-4387"


In [5]:
# parse state abbreviation from address column text
customer_df['state_abbr'] = customer_df.address.str.extract(r'((?:\s)[A-Z]{2}(?:\s))')
customer_df['state_abbr'] = customer_df.state_abbr.str.strip()

In [6]:
customer_df.head()

,account,revenue,employees,address,state_abbr
0,Sunnamplex,4592.96,13938.0,"Apt. 209 94528 Carleton Route, Jaidenchester, IL 74994-0588",IL
1,Silis,5339.57,18053.0,"Suite 103 106 Franecki Pines, Port Genesis, IL 28566",IL
2,Groovestreet,2728.86,6486.0,"1647 Auer Views, Dinabury, IL 44654-4387",IL
3,Donware,2009.52,3409.0,"94578 Tate Springs, Port Betty, IL 03977",IL
4,Wonka Industries,4962.27,4687.0,"Suite 030 4268 Octavia Vista, East Fabianborough, TX 10083-2869",TX


In [13]:
irs_df.head(3)

,STATE,ZIPCODE,Number of returns,Adjusted gross income (AGI),Avg AGI,Number of returns with total income,Total income amount,Avg total income,Number of returns with taxable income,Taxable income amount,Avg taxable income
0,AL,0,2022380,105089761,51.963410,2022380,106420533,52.621433,1468370,67850874,46.208295
1,AL,35004,4930,255534,51.832454,4930,258024,52.337525,4020,163859,40.760945
2,AL,35005,3300,128387,38.905152,3300,129390,39.209091,2440,70760,29.000000


### Insert "real" zip codes

This block of code substitutes real zip codes for the fake ones in our fake customer addresses. This will enable us to join on the zip_code column, and demonstrate linked data functionality.

In [7]:
# randomly select some real zip-codes to replace the "fake" zip-codes
irs_df = pd.read_csv('https://query.data.world/s/slzbv54vahbxdpfaxhdqirphll2bh2')

sample_zips = []
for i in range(97):
    sample_zips.append(irs_df[irs_df.STATE == customer_df.state_abbr[i]].ZIPCODE.sample(1).values[0])
sample_zips = pd.Series(sample_zips).astype(str).str.pad(width=5, fillchar='0')

count = 0
for i in range(97):
    customer_df.loc[i, 'address'] = re.sub(r'(?:\s|-|\b)([\d]{5})(?:\s)',  
                                           " " + str(sample_zips[i])[::-1] + " ", 
                                           customer_df.loc[i, 'address'][::-1], count=1)[::-1]

# parse zip code from address column text
customer_df['zip_code'] = customer_df.address.apply(lambda x: str(x).split(','))
customer_df['zip_code'] = customer_df.zip_code.apply(lambda x: re.findall(r'(?:\s)([\d]{5})(?:\s|-|\b)', x[-1] )[0])

# look at first 3 rows
customer_df.head(3)

### upload the transformed dataframe into our project in data.world
specify the project to upload the data file to    
eg, `project = 'ddw-biz-team/guided-eval-project'`

In [12]:
# specify location to upload to (in this case, it's a project)
project = ''

# upload the dataframe as a csv using the data.world Python library
with ddw.open_remote_file(project, 'accounts_parsed.csv', mode='w') as f:
    customer_df.to_csv(f, index=False)
